In [ ]:
import hoomd
import numpy as np

from monk import prep, pair

from numba import njit

In [ ]:

class CustomAction(hoomd.custom.Action):

    def __init__(self):
        pass

    def act(self, timestep):
        with self._state.cpu_local_snapshot as data:
            pos = data.particles.position._coerce_to_ndarray()
            rtags = data.particles.rtag._coerce_to_ndarray()
            tags = data.particles.tag._coerce_to_ndarray()
            print(f"\n\n\n{timestep}")
            print(pos[24])
            print(rtags[24])
            print(tags[24])

In [ ]:
seed = 1000
device = hoomd.device.CPU()
sim = hoomd.Simulation(device, seed=seed)
N = 64
rng = prep.init_rng(seed + 1)
L = prep.len_from_phi(N, 1.2, dim=2)
snap = prep.approx_euclidean_snapshot(N, L, rng, dim=2, ratios=[60, 40], diams=[1.0, 0.88])

print(snap.particles.position[:10])

sim.create_state_from_snapshot(snap)

In [ ]:
integrator = hoomd.md.Integrator(dt=0.0025)
tree = hoomd.md.nlist.Tree(0.3)
lj = pair.KA_LJ(tree)
nvt = hoomd.md.methods.NVT(hoomd.filter.All(), 1.0, 0.25)
integrator.forces = [lj]
integrator.methods = [nvt]

sim.operations.integrator = integrator

sim.run(0)

In [ ]:
trigger = hoomd.trigger.Periodic(100)
remove_drift = hoomd.update.CustomUpdater(trigger, CustomAction())
sim.operations.updaters.clear()
sim.operations.updaters.append(remove_drift)

In [ ]:
remove_drift.act(sim.timestep)

In [ ]:
sim.run(1000)

In [ ]:
sim.state.get_snapshot().particles.position[59]

In [ ]:
sim.state.get_snapshot()._cpp_obj.particles.position[59]

In [ ]:
sim.state.box._cpp_obj.minImage(10)

In [ ]:
@njit
def test(wrap_func):
    print(wrap_func([1.0, 2.0, 3.0]))

In [ ]:
test(sim.state.box._cpp_obj.minImage)